# Wrangling Spark!

##### This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

In [19]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder.getOrCreate()

### 1. Read the case, department, and source data into their own spark dataframes.

In [2]:
case = spark.read.csv("case.csv", sep=",", header=True, inferSchema=True)
dept = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)
source = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

In [3]:
case.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
-RECORD 1----------------------------------------------------
 case_id

In [4]:
dept.show(2, vertical=True, truncate=False)

-RECORD 0----------------------------------------
 dept_division          | 311 Call Center        
 dept_name              | Customer Service       
 standardized_dept_name | Customer Service       
 dept_subject_to_SLA    | YES                    
-RECORD 1----------------------------------------
 dept_division          | Brush                  
 dept_name              | Solid Waste Management 
 standardized_dept_name | Solid Waste            
 dept_subject_to_SLA    | YES                    
only showing top 2 rows



In [5]:
source.show(2, vertical=True, truncate=False)

-RECORD 0---------------------------
 source_id       | 100137           
 source_username | Merlene Blodgett 
-RECORD 1---------------------------
 source_id       | 103582           
 source_username | Carmen Cura      
only showing top 2 rows



### 2. Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
- Inspect your folder structure. What do you notice?

In [6]:
case.write.json("case_json", mode="overwrite")
case.write.csv("case_csv", mode="overwrite")

### 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [20]:
case.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: string (nullable = true)
 |-- case_closed_date: string (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)



In [8]:
case.explain()

== Physical Plan ==
FileScan csv [case_id#16,case_opened_date#17,case_closed_date#18,SLA_due_date#19,case_late#20,num_days_late#21,case_closed#22,dept_division#23,service_request_type#24,SLA_days#25,case_status#26,source_id#27,request_address#28,council_district#29] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/jeffakins/codeup data science/spark/case.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<case_id:int,case_opened_date:string,case_closed_date:string,SLA_due_date:string,case_late:...




In [9]:
dept.printSchema()

root
 |-- dept_division: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)



In [10]:
source.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- source_username: string (nullable = true)



In [25]:
# Joining the data frames:
df = case.join(dept, 'dept_division', 'left').join(source, 'source_id', 'left')

In [26]:
case.show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------
 case_id              | 1014127332                           
 case_opened_date     | 1/1/18 0:42                          
 case_closed_date     | 1/1/18 12:29                         
 SLA_due_date         | 9/26/20 0:42                         
 case_late            | NO                                   
 num_days_late        | -998.5087616000001                   
 case_closed          | YES                                  
 dept_division        | Field Operations                     
 service_request_type | Stray Animal                         
 SLA_days             | 999.0                                
 case_status          | Closed                               
 source_id            | svcCRMLS                             
 request_address      | 2315  EL PASO ST, San Antonio, 78207 
 council_district     | 5                                    
only showing top 1 row



In [27]:
# Comparing 'case' df above to the joined df below:
df.show(2, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 1/1/18 0:42                          
 case_closed_date       | 1/1/18 12:29                         
 SLA_due_date           | 9/26/20 0:42                         
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 dept_name              | Animal Care Se

In [28]:
# Changing date to datetime
df = (
    df.withColumn('case_opened_date', F.to_timestamp(F.col('case_opened_date'), 'M/d/yy H:mm'))
    .withColumn('case_closed_date', F.to_timestamp(F.col('case_closed_date'), 'M/d/yy H:mm'))
    .withColumn('SLA_due_date', F.to_timestamp(F.col('SLA_due_date'), 'M/d/yy H:mm'))
)

In [29]:
df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: integer (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- source_username: string (nullable = true)



## Data Questions

### 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [37]:
(
    df.select(F.datediff(F.current_timestamp(), 'SLA_due_date')
    .alias('days_past_due'))
    .where(df.case_closed == 'NO')
    .sort(F.col('days_past_due').desc())
    .show(5)
)

+-------------+
|days_past_due|
+-------------+
|         1757|
|         1757|
|         1757|
|         1756|
|         1754|
+-------------+
only showing top 5 rows



### 2. How many Stray Animal cases are there?

In [44]:
(
    df.filter(df.service_request_type == 'Stray Animal')
    .count()
)

27361

### 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [45]:
(
    df.filter(df.dept_division == 'Field Operations')
    .filter(df.service_request_type != 'Officer Standby')
    .count()
)

116295

### 4. Convert the council_district column to a string column.

In [48]:
df.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 12:29:00                  
 SLA_due_date           | 2020-09-26 00:42:00                  
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 dept_name              | Animal Care Se

In [50]:
# Converting to string type:
df = df.withColumn("council_district", F.col("council_district").cast("string"))

In [51]:
df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- source_username: string (nullable = true)



### 5. Extract the year from the case_closed_date column.

In [56]:
df.select(df.case_closed_date, F.year(df.case_closed_date)).show(5)

+-------------------+----------------------+
|   case_closed_date|year(case_closed_date)|
+-------------------+----------------------+
|2018-01-01 12:29:00|                  2018|
|2018-01-03 08:11:00|                  2018|
|2018-01-02 07:57:00|                  2018|
|2018-01-02 08:13:00|                  2018|
|2018-01-01 13:29:00|                  2018|
+-------------------+----------------------+
only showing top 5 rows



### 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [59]:
df.select(df.num_days_late, (df.num_days_late * 24).alias('num_of_hours_late')).show(5)

+-------------------+-------------------+
|      num_days_late|  num_of_hours_late|
+-------------------+-------------------+
| -998.5087616000001|     -23964.2102784|
|-2.0126041669999997|-48.302500007999996|
|       -3.022337963|      -72.536111112|
|       -15.01148148|      -360.27555552|
|0.37216435200000003|  8.931944448000001|
+-------------------+-------------------+
only showing top 5 rows



### 7. Join the case data with the source and department data.

In [60]:
# Completed above
df.printSchema()

root
 |-- source_id: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: timestamp (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- dept_name: string (nullable = true)
 |-- standardized_dept_name: string (nullable = true)
 |-- dept_subject_to_SLA: string (nullable = true)
 |-- source_username: string (nullable = true)



### 8. Are there any cases that do not have a request source?

In [62]:
df.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 12:29:00                  
 SLA_due_date           | 2020-09-26 00:42:00                  
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 dept_name              | Animal Care Se

In [76]:
df.filter(F.isnull(df.source_id)).count()

0

In [69]:
df.select(df.source_id).count()

855269

### 9. What are the top 10 service request types in terms of number of requests?

In [92]:
(
    df.select(df.service_request_type)
    .groupBy(df.service_request_type)
    .count().sort(F.col('count').desc()).show(10, truncate=False)
)

+--------------------------------+-----+
|service_request_type            |count|
+--------------------------------+-----+
|No Pickup                       |89210|
|Overgrown Yard/Trash            |66403|
|Bandit Signs                    |32968|
|Damaged Cart                    |31163|
|Front Or Side Yard Parking      |28920|
|Stray Animal                    |27361|
|Aggressive Animal(Non-Critical) |25492|
|Cart Exchange Request           |22608|
|Junk Vehicle On Private Property|21649|
|Pot Hole Repair                 |20827|
+--------------------------------+-----+
only showing top 10 rows



### 10. What are the top 10 service request types in terms of average days late?

In [99]:
(
    df.where(df.case_late == 'YES') 
    .groupBy(df.service_request_type)
    .agg(F.mean(df.num_days_late).alias('n_days_late'), F.count('*').alias('n_cases'))
    .sort(F.desc('n_days_late'))
    .show(10, truncate=False)
)

+--------------------------------------+------------------+-------+
|service_request_type                  |n_days_late       |n_cases|
+--------------------------------------+------------------+-------+
|Zoning: Recycle Yard                  |210.89201994318182|132    |
|Zoning: Junk Yards                    |200.2051760849428 |262    |
|Structure/Housing Maintenance         |190.20707698509807|51     |
|Donation Container Enforcement        |171.09115313942624|122    |
|Storage of Used Mattress              |163.96812829714287|7      |
|Labeling for Used Mattress            |162.43032902285717|7      |
|Record Keeping of Used Mattresses     |153.99724039428568|7      |
|Signage Requied for Sale of Used Mattr|151.63868055333333|12     |
|Traffic Signal Graffiti               |137.64583330000002|4      |
|License Requied Used Mattress Sales   |128.79828704142858|7      |
+--------------------------------------+------------------+-------+
only showing top 10 rows



### 11. Does number of days late depend on department?

In [ ]:
(
    df.filter('case_late')
    .groupby('dept_name')
    .agg(mean('num_days_late').alias('days_late'), count('num_days_late').alias('n_cases_late'))
    .sort('days_late')
    .withColumn('days_late', round(col('days_late'), 1))
    .show(truncate=False)
)

### 12. How do number of days late depend on department and request type?

In [103]:
df.show(1, vertical=True, truncate=False)

-RECORD 0------------------------------------------------------
 source_id              | svcCRMLS                             
 dept_division          | Field Operations                     
 case_id                | 1014127332                           
 case_opened_date       | 2018-01-01 00:42:00                  
 case_closed_date       | 2018-01-01 12:29:00                  
 SLA_due_date           | 2020-09-26 00:42:00                  
 case_late              | NO                                   
 num_days_late          | -998.5087616000001                   
 case_closed            | YES                                  
 service_request_type   | Stray Animal                         
 SLA_days               | 999.0                                
 case_status            | Closed                               
 request_address        | 2315  EL PASO ST, San Antonio, 78207 
 council_district       | 5                                    
 dept_name              | Animal Care Se

In [130]:
(
    df.groupBy(df.dept_division).agg(F.avg(df.num_days_late).alias('days_late')).sort(F.desc('days_late')).show()
)

+--------------------+-------------------+
|       dept_division|          days_late|
+--------------------+-------------------+
|Code Enforcement ...| 135.92851612479797|
|        Reservations|        66.03116319|
|     311 Call Center| 59.737091496300785|
|Director's Office...|  37.57064670295009|
|Engineering Division| 13.433724555869722|
|               Shops|   9.64126176872269|
|           Tree Crew|  4.723282812065399|
|         Solid Waste| 3.5190239198762256|
|              Trades| 3.2319771412769325|
|Clean and Green N...| 1.6914689194878048|
|              Vector| -1.120653299322344|
|    Facility License| -1.412693770221664|
|       Miscellaneous|-1.7218576838926682|
|    Waste Collection| -2.170652238479894|
|     Clean and Green|-2.5571549792541437|
|               Brush|-3.9857905714570965|
| Food Establishments| -6.971552370451527|
|  Signs and Markings| -7.448628001357719|
|    Shops (Internal)| -8.186267119876456|
|         Storm Water|-14.055678397031896|
+----------